In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import sklearn
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import GridSearchCV

import tensorflow as tf
from tensorflow import data

np.random.seed(0)
tf.random.set_seed(0)

In [2]:
!git clone https://github.com/MLinApp-FP01-Team7-24/OurProject.git
%cd OurProject

Cloning into 'OurProject'...
remote: Enumerating objects: 361, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 361 (delta 7), reused 16 (delta 3), pack-reused 339
Receiving objects: 100% (361/361), 18.70 MiB | 8.36 MiB/s, done.
Resolving deltas: 100% (105/105), done.
/content/OurProject


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ROOTDIR_DATASET_NORMAL = "/content/drive/MyDrive/normal"
ROOTDIR_DATASET_COLLISION = "/content/drive/MyDrive/collisions"

Mounted at /content/drive


In [4]:
window_size = 20
k_pa = 0 # valori tra 0 e 1

In [8]:
from Models.LSTM_VAE.data import get_test_windows, get_cal_windows

# data_cal, label_cal = get_cal_windows(window_size, k_pa)
data_test, label_test = get_test_windows(window_size, k_pa, filepath=ROOTDIR_DATASET_COLLISION)

# print(data_cal.shape, label_cal.shape)
print(data_test.shape, label_test.shape)

(34255, 20, 55) (34255,)


In [10]:
batch_size = 64
x_dim = data_cal.shape[2]
lstm_h_dim = 10
z_dim = 10

NameError: name 'data_cal' is not defined

In [6]:
from Models.LSTM_VAE.model import LSTM_VAE

model = LSTM_VAE(window_size, x_dim, lstm_h_dim, z_dim, dtype='float32')
model.load_weights('./trained_models/LSTM_VAE/checkpoint')

NameError: name 'x_dim' is not defined

In [ ]:
X_cal = tf.convert_to_tensor(data_cal, dtype='float32')
cal_dataset = data.Dataset.from_tensor_slices(X_cal)
cal_dataset = cal_dataset.shuffle(buffer_size=1024).batch(batch_size, drop_remainder=True)

y_pred_cal = model.predict(cal_dataset)
score_cal = np.array((data_cal[:y_pred_cal.shape[0]] - y_pred_cal) ** 2).mean(1)
label_cal = label_cal[:score_cal.shape[0]]

In [ ]:
optimize = True
param_grid = {'estimator__C': np.logspace(0, 5, 6)}

if optimize:
  cal_search = GridSearchCV(CalibratedClassifierCV(SVC(probability=True), cv=3), param_grid, cv=3, verbose=1, scoring='roc_auc')
  cal_search.fit(score_cal, label_cal)
  cal_model = cal_search.best_estimator_
  print(cal_search.best_params_)
else:
  cal_model = CalibratedClassifierCV(SVC(probability=True, C=1000))
  cal_model.fit(score_cal, label_cal[:score_cal.shape[0]])

In [ ]:
X_eval = tf.convert_to_tensor(data_eval, dtype='float32')
eval_dataset = data.Dataset.from_tensor_slices(X_eval)
eval_dataset = eval_dataset.batch(batch_size, drop_remainder=True)

y_pred_eval = model.predict(eval_dataset)
score_eval = np.array((data_eval[:y_pred_eval.shape[0]] - y_pred_eval) ** 2).mean(1)
label_eval = label_eval[:score_eval.shape[0]]

y_pred_eval = cal_model.predict(score_eval)
y_score_eval = cal_model.predict_proba(score_eval)[:, 1]

In [ ]:
f1 = sklearn.metrics.f1_score(label_eval, y_pred_eval)
f1

In [ ]:
fpr, tpr, thresholds = sklearn.metrics.roc_curve(label_eval, y_score_eval)
roc_auc = sklearn.metrics.auc(fpr, tpr)

plt.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % roc_auc)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend()
plt.show()

roc_auc

In [ ]:
precison, recall, thresholds = sklearn.metrics.precision_recall_curve(label_eval, y_score_eval)
prc_auc = sklearn.metrics.auc(recall, precison)

plt.plot(recall, precison, label='PRC curve (area = %0.4f)' % prc_auc)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision Recall Curve')
plt.legend()
plt.show()

prc_auc